# Training Pipeline
[run_training_pipeline.ipynb](https://github.com/shibing624/MedicalGPT/blob/main/run_training_pipeline.ipynb)    | [Open In Colab](https://colab.research.google.com/github/shibing624/MedicalGPT/blob/main/run_training_pipeline.ipynb)

# Stage 1: Continue Pretraining

第一阶段：PT(Continue PreTraining)增量预训练，在海量领域文本数据上二次预训练GPT模型，以适配领域数据分布

注意：
1. 此阶段是可选的，如果你没有海量领域文本，可以跳过此阶段，直接进行SFT阶段的有监督微调
2. 我实验发现：做领域知识注入，SFT比PT更高效，也可以跳过PT阶段

| Stage 1: Continue Pretraining   |  [pretraining.py](https://github.com/shibing624/MedicalGPT/blob/main/pretraining.py) | [run_pt.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_pt.sh)    |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Qwen/Qwen2.5-0.5B
2. 数据集：PT阶段使用的是中文天龙八部小说部分文本和英文书籍部分文本，位于`data/pretrain`文件夹

## 配置运行环境

本地执行可注释以下配置环境的命令，colab执行要打开注释，用于配置环境

colab建议使用T4 GPU训练，设置方式：`代码执行程序 -> 更改运行时类型 -> 运行时类型：Python3，硬件加速器：GPU，GPU类型：T4 -> 保存`

步骤：
1. 下载最新代码到本地
2. 安装依赖包

依赖包如下，保证最新版本：

```
loguru
transformers
sentencepiece
datasets
tensorboard
tqdm
peft
trl
```

In [1]:
!git clone --depth 1 https://github.com/shibing624/MedicalGPT.git
%cd MedicalGPT
%ls
!pip install -r requirements.txt

Cloning into 'MedicalGPT'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 98 (delta 19), reused 52 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (98/98), 8.98 MiB | 16.36 MiB/s, done.
Resolving deltas: 100% (19/19), done.
/content/MedicalGPT
build_domain_tokenizer.py   requirements.txt
chatpdf.py                  reward_modeling.py
CITATION.cff                role_play_data/
_config.yml                 run_dpo.sh
CONTRIBUTING.md             run_eval_quantize.sh
convert_dataset.py          run_full_sft.sh
data/                       run_grpo.sh
DISCLAIMER                  run_orpo.sh
docs/                       run_ppo.sh
dpo_training.py             run_pt.sh
eval_quantize.py            run_quant.sh
fastapi_server_demo.py      run_rm.sh
gradio_demo.py              run_sft_accelerate.sh
grpo_training.py            run_sft.sh
inference_multigpu_demo.py  run_training_dpo_pipel

## Stage1 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

**以下参数可以根据你的GPU实际情况修改，当前参数是根据Colab的T4单卡GPU（16GB显存）配置的**

In [2]:
%ls ./data/pretrain/

en_article_tail500.txt  fever.txt  tianlongbabu.txt


In [4]:
!python pretraining.py \
    --model_name_or_path Qwen/Qwen2.5-0.5B \
    --train_file_dir ./data/pretrain \
    --validation_file_dir ./data/pretrain \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 3 \
    --do_train \
    --do_eval \
    --use_peft True \
    --seed 42 \
    --bf16 \
    --max_train_samples 20000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-4 \
    --warmup_ratio 0.05 \
    --weight_decay 0.01 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --eval_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --block_size 128 \
    --group_by_length True \
    --output_dir outputs-pt-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype bfloat16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

2025-12-23 04:56:35.922352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766465795.953787    1387 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766465795.963354    1387 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766465795.989721    1387 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766465795.989755    1387 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766465795.989763    1387 computation_placer.cc:177] computation placer alr

In [5]:
%ls -lh outputs-pt-v1

total 22M
-rw-r--r-- 1 root root 1.1K Dec 23 05:06 adapter_config.json
-rw-r--r-- 1 root root  17M Dec 23 05:06 adapter_model.safetensors
-rw-r--r-- 1 root root  605 Dec 23 05:06 added_tokens.json
-rw-r--r-- 1 root root  472 Dec 23 05:06 all_results.json
-rw-r--r-- 1 root root 2.4K Dec 23 05:06 chat_template.jinja
drwxr-xr-x 2 root root 4.0K Dec 23 05:03 checkpoint-500/
drwxr-xr-x 2 root root 4.0K Dec 23 05:06 checkpoint-834/
-rw-r--r-- 1 root root  263 Dec 23 05:06 eval_results.json
-rw-r--r-- 1 root root 1.6M Dec 23 05:06 merges.txt
-rw-r--r-- 1 root root 5.1K Dec 23 05:06 README.md
drwxr-xr-x 3 root root 4.0K Dec 23 04:57 runs/
-rw-r--r-- 1 root root  616 Dec 23 05:06 special_tokens_map.json
-rw-r--r-- 1 root root 4.7K Dec 23 05:06 tokenizer_config.json
-rw-r--r-- 1 root root  20K Dec 23 05:06 trainer_state.json
-rw-r--r-- 1 root root  229 Dec 23 05:06 train_results.json
-rw-r--r-- 1 root root 3.3M Dec 23 05:06 vocab.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.safetensors`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [6]:
!python merge_peft_adapter.py \
    --base_model Qwen/Qwen2.5-0.5B --lora_model outputs-pt-v1 --output_dir merged-pt/

2025-12-23 05:18:58.167603: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766467138.188154    7181 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766467138.194388    7181 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766467138.210785    7181 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766467138.210820    7181 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766467138.210826    7181 computation_placer.cc:177] computation placer alr

In [7]:
%ls -lh merged-pt/

total 958M
-rw-r--r-- 1 root root  605 Dec 23 05:19 added_tokens.json
-rw-r--r-- 1 root root 2.4K Dec 23 05:19 chat_template.jinja
-rw-r--r-- 1 root root 1.3K Dec 23 05:19 config.json
-rw-r--r-- 1 root root  117 Dec 23 05:19 generation_config.json
-rw-r--r-- 1 root root 1.6M Dec 23 05:19 merges.txt
-rw-r--r-- 1 root root 943M Dec 23 05:19 model.safetensors
-rw-r--r-- 1 root root  616 Dec 23 05:19 special_tokens_map.json
-rw-r--r-- 1 root root 4.6K Dec 23 05:19 tokenizer_config.json
-rw-r--r-- 1 root root  11M Dec 23 05:19 tokenizer.json
-rw-r--r-- 1 root root 2.7M Dec 23 05:19 vocab.json


In [8]:
%cat merged-pt/config.json

{
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention"
  ],
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1

Stage1 增量预训练完成。

# Stage 2: Supervised FineTuning

第二阶段：SFT(Supervised Fine-tuning)有监督微调，构造指令微调数据集，在预训练模型基础上做指令精调，以对齐指令意图，并注入领域知识

| Stage 2: Supervised Fine-tuning | [supervised_finetuning.py](https://github.com/shibing624/MedicalGPT/blob/main/supervised_finetuning.py) | [run_sft.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_sft.sh)  |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Qwen/Qwen2.5-0.5B 或者 Stage1得到的预训练模型
2. 数据集：SFT阶段使用的是使用的是Belle的1千条抽样数据，位于`data/finetune`文件夹

## Stage2 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [9]:
%ls ./data/finetune

medical_sft_1K_format.jsonl        sharegpt_zh_1K_format.jsonl
numina_cot_sharegpt_data_1k.jsonl


In [10]:
# 1) 备份原来的（含 id 的）文件
!mv ./data/finetune/numina_cot_sharegpt_data_1k.jsonl ./data/finetune/numina_cot_sharegpt_data_1k.jsonl.bak

# 2) 用 strict 版本替换成脚本会读到的文件名
!cp ./data/finetune/numina_cot_sharegpt_data_1k.strict.jsonl ./data/finetune/numina_cot_sharegpt_data_1k.jsonl

# 3) 自检：确认现在目录里已经是“替换后的”
!ls -lh ./data/finetune/numina_cot_sharegpt_data_1k.jsonl*

cp: cannot stat './data/finetune/numina_cot_sharegpt_data_1k.strict.jsonl': No such file or directory
-rw-r--r-- 1 root root 1.6M Dec 23 04:54 ./data/finetune/numina_cot_sharegpt_data_1k.jsonl.bak


In [11]:
!python supervised_finetuning.py \
    --model_name_or_path merged-pt \
    --train_file_dir ./data/finetune \
    --validation_file_dir ./data/finetune \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --bf16 \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-5 \
    --warmup_ratio 0.05 \
    --weight_decay 0.05 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --eval_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --output_dir outputs-sft-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype bfloat16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

2025-12-23 05:20:36.576044: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766467236.597086    7623 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766467236.603382    7623 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766467236.618836    7623 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766467236.618866    7623 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766467236.618870    7623 computation_placer.cc:177] computation placer alr

In [12]:
%ls -lh outputs-sft-v1

total 22M
-rw-r--r-- 1 root root 1.1K Dec 23 05:32 adapter_config.json
-rw-r--r-- 1 root root  17M Dec 23 05:32 adapter_model.safetensors
-rw-r--r-- 1 root root  605 Dec 23 05:32 added_tokens.json
-rw-r--r-- 1 root root  430 Dec 23 05:32 all_results.json
-rw-r--r-- 1 root root 2.4K Dec 23 05:32 chat_template.jinja
drwxr-xr-x 2 root root 4.0K Dec 23 05:32 checkpoint-250/
-rw-r--r-- 1 root root  221 Dec 23 05:32 eval_results.json
-rw-r--r-- 1 root root 1.6M Dec 23 05:32 merges.txt
-rw-r--r-- 1 root root 5.1K Dec 23 05:32 README.md
drwxr-xr-x 3 root root 4.0K Dec 23 05:21 runs/
-rw-r--r-- 1 root root  648 Dec 23 05:32 special_tokens_map.json
-rw-r--r-- 1 root root 4.7K Dec 23 05:32 tokenizer_config.json
-rw-r--r-- 1 root root 6.0K Dec 23 05:32 trainer_state.json
-rw-r--r-- 1 root root  229 Dec 23 05:32 train_results.json
-rw-r--r-- 1 root root 3.3M Dec 23 05:32 vocab.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.safetensors`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [13]:
!python merge_peft_adapter.py \
    --base_model merged-pt --lora_model outputs-sft-v1 --output_dir merged-sft/

2025-12-23 05:34:19.242958: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766468059.263257   11125 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766468059.269321   11125 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766468059.284689   11125 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766468059.284722   11125 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766468059.284730   11125 computation_placer.cc:177] computation placer alr

In [14]:
%ls -lh merged-sft/

total 958M
-rw-r--r-- 1 root root  605 Dec 23 05:34 added_tokens.json
-rw-r--r-- 1 root root 2.4K Dec 23 05:34 chat_template.jinja
-rw-r--r-- 1 root root 1.3K Dec 23 05:34 config.json
-rw-r--r-- 1 root root  117 Dec 23 05:34 generation_config.json
-rw-r--r-- 1 root root 1.6M Dec 23 05:34 merges.txt
-rw-r--r-- 1 root root 943M Dec 23 05:34 model.safetensors
-rw-r--r-- 1 root root  616 Dec 23 05:34 special_tokens_map.json
-rw-r--r-- 1 root root 4.6K Dec 23 05:34 tokenizer_config.json
-rw-r--r-- 1 root root  11M Dec 23 05:34 tokenizer.json
-rw-r--r-- 1 root root 2.7M Dec 23 05:34 vocab.json


In [15]:
%cat merged-sft/config.json

{
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention"
  ],
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1

Stage2 SFT训练完成。

# Stage 3: Reward Modeling

第三阶段：RM(Reward Model)奖励模型建模，构造人类偏好排序数据集，训练奖励模型，用来对齐人类偏好，主要是"HHH"原则，具体是"helpful, honest, harmless"

| Stage 3: Reward Modeling        |  [reward_modeling.py](https://github.com/shibing624/MedicalGPT/blob/main/reward_modeling.py) | [run_rm.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_rm.sh)    |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Qwen/Qwen2.5-0.5B 或者 Stage2得到的SFT模型
2. 数据集：RM阶段使用的是医疗reward数据，抽样了500条，位于`data/reward`文件夹

## Stage3 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [16]:
%ls ./data/reward/

dpo_zh_500.jsonl


In [17]:
!python reward_modeling.py \
    --model_name_or_path merged-sft \
    --train_file_dir ./data/reward \
    --validation_file_dir ./data/reward \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --do_train \
    --use_peft True \
    --seed 42 \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-5 \
    --warmup_ratio 0.05 \
    --weight_decay 0.001 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --eval_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --max_source_length 256 \
    --max_target_length 256 \
    --output_dir outputs-rm-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --fp16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --remove_unused_columns False \
    --gradient_checkpointing True

2025-12-23 05:35:24.835184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766468124.874660   11418 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766468124.886410   11418 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766468124.914210   11418 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766468124.914250   11418 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766468124.914254   11418 computation_placer.cc:177] computation placer alr

In [18]:
%ls -lh outputs-rm-v1

total 22M
-rw-r--r-- 1 root root 1.1K Dec 23 05:39 adapter_config.json
-rw-r--r-- 1 root root  17M Dec 23 05:39 adapter_model.safetensors
-rw-r--r-- 1 root root  605 Dec 23 05:39 added_tokens.json
-rw-r--r-- 1 root root  484 Dec 23 05:39 all_results.json
-rw-r--r-- 1 root root 2.4K Dec 23 05:39 chat_template.jinja
drwxr-xr-x 2 root root 4.0K Dec 23 05:39 checkpoint-339/
-rw-r--r-- 1 root root  291 Dec 23 05:39 eval_results.json
-rw-r--r-- 1 root root 1.6M Dec 23 05:39 merges.txt
-rw-r--r-- 1 root root 5.1K Dec 23 05:39 README.md
drwxr-xr-x 3 root root 4.0K Dec 23 05:36 runs/
-rw-r--r-- 1 root root  648 Dec 23 05:39 special_tokens_map.json
-rw-r--r-- 1 root root 4.6K Dec 23 05:39 tokenizer_config.json
-rw-r--r-- 1 root root 8.4K Dec 23 05:39 trainer_state.json
-rw-r--r-- 1 root root  213 Dec 23 05:39 train_results.json
-rw-r--r-- 1 root root 3.3M Dec 23 05:39 vocab.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.safetensors`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [19]:
!python merge_peft_adapter.py \
    --base_model merged-sft --lora_model outputs-rm-v1 --output_dir merged-rm/

2025-12-23 05:42:47.052192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766468567.085106   13372 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766468567.093194   13372 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766468567.117168   13372 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766468567.117207   13372 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766468567.117215   13372 computation_placer.cc:177] computation placer alr

In [20]:
%ls -lh merged-rm/

total 1.9G
-rw-r--r-- 1 root root  605 Dec 23 05:42 added_tokens.json
-rw-r--r-- 1 root root 2.4K Dec 23 05:42 chat_template.jinja
-rw-r--r-- 1 root root 1.4K Dec 23 05:42 config.json
-rw-r--r-- 1 root root 1.6M Dec 23 05:42 merges.txt
-rw-r--r-- 1 root root 1.9G Dec 23 05:43 model.safetensors
-rw-r--r-- 1 root root  616 Dec 23 05:42 special_tokens_map.json
-rw-r--r-- 1 root root 4.6K Dec 23 05:42 tokenizer_config.json
-rw-r--r-- 1 root root  11M Dec 23 05:42 tokenizer.json
-rw-r--r-- 1 root root 2.7M Dec 23 05:42 vocab.json


In [21]:
%cat merged-rm/config.json

{
  "architectures": [
    "Qwen2ForSequenceClassification"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "float32",
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 896,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention"
  ],
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_atten

Stage3 奖励建模第一次训练完成。

# Stage 4: Reinforcement Learning Training

第四阶段：RL(Reinforcement Learning)基于人类反馈的强化学习(RLHF)，用奖励模型来训练SFT模型，生成模型使用奖励或惩罚来更新其策略，以便生成更高质量、更符合人类偏好的文本

| Stage 4: Reinforcement Learning |  [rl_training.py](https://github.com/shibing624/MedicalGPT/blob/main/rl_training.py) | [run_rl.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_rl.sh)    |


#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型、奖励模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Qwen/Qwen2.5-0.5B 或者 Stage2得到的SFT模型
2. 奖励模型：使用的是`OpenAssistant/reward-model-deberta-v3-large-v2` 或者 Stage3得到的BERT类或者GPT类奖励模型
3. 数据集：RL阶段的数据可以复用SFT的数据集，使用的是Belle的1千条抽样数据，位于`data/finetune`文件夹

## Stage4 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载生成模型和tokenizer，加载奖励模型和其tokenizer
5. 开始训练并评估
6. 查看训练结果

以下参数可以根据你的GPU实际情况修改，当前参数是根据Colab的T4单卡GPU（16GB显存）配置的。

In [22]:
%ls ./data/finetune/

medical_sft_1K_format.jsonl            sharegpt_zh_1K_format.jsonl
numina_cot_sharegpt_data_1k.jsonl.bak


In [23]:
! PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True CUDA_VISIBLE_DEVICES=0 python ppo_training.py \
  --sft_model_path ./merged-sft \
  --reward_model_path ./merged-rm \
  --template_name qwen \
  --torch_dtype float16 \
  --fp16 True \
  --load_in_8bit True \
  --train_file_dir ./data/finetune \
  --validation_file_dir ./data/finetune \
  --max_source_length 256 \
  --response_length 64 \
  --do_train \
  --num_train_epochs 1 \
  --save_steps 50 \
  --output_dir outputs-ppo-v1 \
  --per_device_train_batch_size 1 \
  --local_rollout_forward_batch_size 1 \
  --micro_batch_size 1 \
  --mini_batch_size 1 \
  --gradient_checkpointing True \
  --load_in_8bit True \
  --report_to tensorboard

2025-12-23 05:44:01.831689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766468641.852003   13697 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766468641.858027   13697 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766468641.873377   13697 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766468641.873429   13697 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766468641.873433   13697 computation_placer.cc:177] computation placer alr

In [ ]:
%ls -lh outputs-ppo-v1

模型训练结果：
- use_peft=False,默认是使用全参训练，模型保存的就是`model-00001-of-00002.safetensors`等文件，配置文件是`config.json`
- use_peft=True, 则使用lora训练模型，则保存的lora权重是`adapter_model.safetensors`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/trl`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/trl --host 0.0.0.0 --port 8009`

In [ ]:
%ls -lh outputs-ppo-v1/

In [ ]:
%cat outputs-ppo-v1/config.json

Stage4 RL第一次训练完成。

**至此一个完整的4阶段训练流程演示完成。**

实际操作中Stage3和Stage4可以反复多次，直到RL得到的最后模型满足评估要求。

RLHF过程可以把SFT模型当成一个初始化模型，RM模型当做指导老师，使用RL(PPO)调教SFT模型生成指导老师最满意的结果，如果小学老师满意了，我们就再训练一个中学老师，继续指导，中学老师满意了，就训练一个大学老师，这样不断迭代，使得生成模型的质量达到甚至超过人工撰写的天花板。

RLHF训练不易，此项目提供给大家一种实现的方法和参考，希望抛砖引玉，共同促进中文开源LLM发展。

# Test

In [ ]:
!python inference.py --base_model merged-ppo-v1
# 或在shell中运行
# !python inference.py --base_model merged-ppo-v1 --interactive

Input:介绍下南京
Response:  南京市位于江苏省西南部，是全国首批历史文化名城、国家中心城市和自由贸易试验区。

完。
